In [ ]:
import pdb

# Read Data

In [ ]:
import numpy as np
import torch
import e3nn

torch.set_default_dtype(torch.float64)
np.set_printoptions(precision=3, suppress=True)

data = np.load("h2o_hamiltonians.npy", allow_pickle=True)
data_dict = data.item()

geometry = torch.from_numpy(data_dict["positions"]).type(
    torch.float64
)  # batch, atom, xyz
hamiltonians = torch.from_numpy(data_dict["hamiltonians"]).type(torch.float64)

hamiltonians.shape

## Parse the output of ORCA

In [ ]:
import torch
import pdb

cnt = 20
geometry = torch.zeros((cnt, 3, 3))
H = torch.zeros(cnt, 24, 24)
for i in range(cnt):
    with open(f"water_new/water_{i}_property.txt") as f:
        lines = f.readlines()[59:62]
        for j in range(3):
            for k in range(3):
                line = lines[j].split(' ')
                line = [item for item in line if not item == '']
                geometry[i, j, k] = float(line[2+k])
    with open(f'water_new/water_{i}.out') as f:
        lines = f.readlines()
        for j, line in enumerate(lines):
            if "Energy Check signals convergence" in line:
                break
        end = j
        for j in range(24):
            for k in range(24):
                row = end - (24-j) - 25*(3 - k//6)
                col = 1 + k%6
                line = lines[row].split(' ')
                line = [item for item in line if not item == '']
                H[i, j, k] = float(line[col])     
                
geometry = geometry.type(torch.float64)
hamiltonians = H.type(torch.float64)

## Parse hdf5

In [19]:
path  = 'h2o_notransform.hdf5'
import h5py
import torch
f = h5py.File(path, "r")
indices = [1113, 2145, 2702, 4229, 4870]
indices = [i for i in range(4999)]
geometry = torch.tensor(f['coord'][indices]).type(torch.float64)
hamiltonians = torch.tensor(f['hamiltonian'][indices]).type(torch.float64).view(-1, 24, 24)
f.close()

In [ ]:
import numpy as np
import e3nn


def null_space(A: np.ndarray, *, epsilon=1e-4, round_fn=lambda x: x) -> np.ndarray:
    r"""
    Compute the null space of a matrix.

    .. math::
        \mathbf{A} \mathbf{X}^T = 0

    Args:
        A: Matrix to compute null space of.
        epsilon: The tolerance for the eigenvalue.

    Returns:
        The null space of A.
    """
    assert A.ndim == 2, "Null space only works for matrices."
    assert A.dtype in [
        np.float64,
        np.complex128,
    ], "Null space only works for float64 matrices."

    # Q, R = np.linalg.qr(A.T)
    # # assert np.allclose(R.T @ Q.T, S)
    # X = Q.T[np.abs(np.diag(R)) < epsilon]
    # X = np.conj(X)

    A = np.conj(A.T) @ A
    A = round_fn(A)
    val, vec = np.linalg.eigh(A)
    X = vec.T[np.abs(val) < epsilon]
    X = np.conj(X.T) @ X
    X = round_fn(X)
    X = gram_schmidt(X, round_fn=round_fn)
    return X


def gram_schmidt(A: np.ndarray, *, epsilon=1e-4, round_fn=lambda x: x) -> np.ndarray:
    """
    Orthogonalize a matrix using the Gram-Schmidt process.
    """
    assert A.ndim == 2, "Gram-Schmidt process only works for matrices."
    assert A.dtype in [
        np.float64,
        np.complex128,
    ], "Gram-Schmidt process only works for float64 matrices."
    Q = []
    for i in range(A.shape[0]):
        v = A[i]
        for w in Q:
            v -= np.dot(np.conj(w), v) * w
        norm = np.linalg.norm(v)
        if norm > epsilon:
            v = round_fn(v / norm)
            Q += [v]
    return np.stack(Q) if len(Q) > 0 else np.empty((0, A.shape[1]))

In [ ]:
# distance_matrix = (geometry.unsqueeze(1) - geometry.unsqueeze(2)).norm(2, -1)
# diff_norm = (distance_matrix.unsqueeze(0) - distance_matrix.unsqueeze(1)).norm(2, -1).norm(2, -1)

In [ ]:
# greater_than_zero = (diff_norm > 1e-12).nonzero()
# greater_than_zero

In [ ]:
# same = (diff_norm[greater_than_zero[:, 0], greater_than_zero[:, 1]] < 1e-3).nonzero()
# same = greater_than_zero[same][0].squeeze()
# # print(same)
# same.shape

In [ ]:
# geometry = geometry[same]
# hamiltonians = hamiltonians[same]
# distance_matrix[same]
# print(geometry)

In [ ]:
import matplotlib.pyplot as plt


def imshow(x, v):
    plt.imshow(x.numpy(), vmin=-v, vmax=v, cmap="bwr")

In [ ]:
scalars = [0, 1, 2, 14 + 0, 14 + 1, 14 + 5 + 0, 14 + 5 + 1]
vectors = [3, 4, 5, 6, 7, 8, 14 + 2, 14 + 3, 14 + 4, 14 + 5 + 2, 14 + 5 + 3, 14 + 5 + 4]
rep5 = [9, 10, 11, 12, 13]

imshow(hamiltonians.std(0)[scalars], 1e-1)
# sss pp d ss p ss p

In [ ]:
def get_abc(pts):
    opts = pts
    pts = pts - pts[0]
    a, b = e3nn.o3.xyz_to_angles(pts[1])
    pts = pts @ e3nn.o3.angles_to_matrix(a, b, torch.tensor(0).type(torch.float64))
    c = torch.atan2(pts[2][0], pts[2][2])
    pts = pts @ e3nn.o3.matrix_y(c)
    
    target = torch.tensor(
        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.5664e-18,  1.0078e+00, -2.6687e-18],
        [-3.7204e-20, -2.1950e-01,  9.2139e-01]]
    ).type(torch.float64) # replace this with a sample from the dataset
    opts = opts - opts[0]
    opts = opts.transpose(1, 0)
    assert torch.allclose(e3nn.o3.angles_to_matrix(a, b, c)@target.transpose(1, 0), opts, atol=1e-3)

    return a, b, c

In [ ]:
def infer_change_of_basis(originals, rotated, reps, eps=1e-5):
    As = []
    for v0, v1, D in zip(originals, rotated, reps):
        d = len(D)
        A = torch.einsum("ij,k->ijk", (D, v0)) - torch.einsum(
            "ij,k->ijk", (torch.eye(d), v1)
        )
        As.append(A.view(d, -1))
    A = torch.cat(As, 0)
    Q = null_space(A.numpy(), epsilon=1e-5) * d ** 0.5
    assert len(Q) == 1, Q.shape
    return Q[0].reshape(d, d)

## Infer the change of basis for l=1

In [ ]:
Rref = e3nn.o3.angles_to_matrix(*get_abc(geometry[0]))

reps = []
for i in range(0, len(geometry)):
    R = (
        e3nn.o3.angles_to_matrix(*get_abc(geometry[i])) @ Rref.T
    )  # rotation from ref -> to i
    a, b, c = e3nn.o3.matrix_to_angles(R)
    reps.append(e3nn.o3.wigner_D(1, a, b, c))

for v in [vectors[:3], vectors[3:6], vectors[6:9], vectors[9:12]]:
    originals = [hamiltonians[0, scalars[1], v]] * len(reps)
    rotated = hamiltonians[:, scalars[1], v]
    Q = infer_change_of_basis(originals, rotated, reps, eps=1e-5)
    print(Q)

## Infer the change of basis for l=2

In [ ]:
Rref = e3nn.o3.angles_to_matrix(*get_abc(geometry[0]))

reps = []
for i in range(0, len(geometry)):
    R = (
        e3nn.o3.angles_to_matrix(*get_abc(geometry[i])) @ Rref.t()
    )  # rotation from ref -> to i
    a, b, c = e3nn.o3.matrix_to_angles(R)
    reps.append(e3nn.o3.wigner_D(2, a, b, c))

originals = [hamiltonians[0, scalars[1], rep5]] * len(reps)
rotated = hamiltonians[:, scalars[1], rep5]
Q = infer_change_of_basis(originals, rotated, reps, eps=1e-5)

print(Q)

In [ ]:
from fractions import Fraction

Fraction(Q[2, 3] ** 2).limit_denominator(100)

# Sanity check

In [ ]:
# sss pp d ss p ss p
S = torch.ones(1, 1)
P = torch.tensor([[0, 1., 0], [0, 0, 1], [1, 0, 0]])
D = torch.tensor(
    [
        [0, 1, 0, 0, 0.0],
        [0, 0, 0, 0, 1],
        [-0.5, 0, 0, -(3 / 4) ** 0.5, 0],
        [0, 0, 1, 0, 0],
        [((3 / 4) ** 0.5), 0, 0, -0.5, 0],
    ]
)

M = e3nn.math.direct_sum(S, S, S, P, P, D, S, S, P, S, S, P)
assert torch.allclose(M.T @ M, torch.eye(M.shape[0]), atol=1e-3)
assert torch.allclose(M @ M.T, torch.eye(M.shape[0]), atol=1e-3)
M=M.type(torch.float64)

In [ ]:
# sss pp d ss p ss p
irreps = e3nn.o3.Irreps("3x0e + 2x1o + 1x2e + 2x0e + 1x1o + 2x0e + 1x1o")

rotated_geometry = []
rotated_hamiltonians = []

for i in range(0, len(geometry)):
    a, b, c = get_abc(geometry[i])
    R = e3nn.o3.angles_to_matrix(a, b, c)
    pos = geometry[i]
    pos = pos - pos[0]
    rotated_geometry.append(pos@R)

    D = irreps.D_from_angles(a, b, c)
    H = M @ hamiltonians[i] @ M.T  # change of basis from hamiltonian's original basis to e3nn basis
    rotated_hamiltonians.append(D.T @ H @ D)  # apply rotation to the hamiltonian

rotated_geometry = torch.stack(rotated_geometry)
rotated_hamiltonians = torch.stack(rotated_hamiltonians)

In [ ]:
# all positions are sent to the same positions => this must be small:
rotated_geometry.std(0)

In [ ]:
# Same for the hamiltonians, this must be small:
rotated_hamiltonians.std(0).abs().max()

In [ ]:
vec = torch.tensor([-1., 0, 0])
a, b = e3nn.o3.xyz_to_angles(vec)